In [19]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import os.path as path
import pandas as pd
import h5py
import numpy as np
from faker import Factory
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly
from ipywidgets import fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from IPython.display import clear_output
from IPython.display import display
from ipywidgets import IntSlider, Output
import plotly.graph_objects as go
import plotly.express as plot_x
import matplotlib.colors as mcolors

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools
from pyccapt.calibration.reconstructions import reconstruction
from pyccapt.calibration.leap_tools import ccapt_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [5]:
button = widgets.Button(
    description='load range dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global range_path
    range_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load range dataset', style=ButtonStyle())

In [8]:
tdc, pulse_mode, flightPathLength, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, flightPathLength)

Dropdown(description='Data mode:', options=('surface_concept', 'roentdec', 'leap_epos', 'leap_pos', 'ato_v6'),…

FloatText(value=110.0, description='Flight path length:')

In [13]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path 
variables.result_data_name = 'reconstruction_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/reconstruction/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

The maximum time of flight: 5010


In [25]:
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,26.987074,0.000122,5263.975586,5263.975586,32552,611.794680,632.252736,-1.572735,-1.661878,0,0
1,0.0,0.0,0.0,26.848762,0.000117,5263.975586,5263.975586,32781,610.402608,619.853472,0.394776,-0.522122,229,2
2,0.0,0.0,0.0,27.716827,0.000122,5263.975586,5263.975586,33039,619.078868,631.765818,1.442204,-0.646286,258,1
3,0.0,0.0,0.0,26.959129,0.000120,5263.975586,5263.975586,33680,611.513719,626.361714,-0.865959,-1.458122,641,1
4,0.0,0.0,0.0,27.008878,0.000120,5263.975586,5263.975586,33732,612.013785,626.828058,1.840163,-0.569878,52,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888681,0.0,0.0,0.0,27.196443,0.000108,5710.897949,5710.897949,27153,613.894888,619.195104,3.158204,0.105061,420,1
1888682,0.0,0.0,0.0,26.996092,0.000107,5710.897949,5710.897949,27190,611.885311,616.959396,3.145469,0.060490,37,1
1888683,0.0,0.0,0.0,26.998240,0.000104,5710.897949,5710.897949,27456,611.906897,608.530914,-1.728735,-0.079592,266,1
1888684,0.0,0.0,0.0,26.800806,0.000101,5710.897949,5710.897949,28120,609.919070,600.321888,1.152490,-1.044245,664,1


In [23]:
if 'range_path' in locals():
    variables.range_data = data_tools.read_hdf5_through_pandas(range_path)

else:
    variables.range_data = pd.DataFrame({"ion": ['unranged'], "mass": [0], "mc": [0], "mass": [0], "mc_low": [0], "mc_up": [0], "color": ['#000000'], "element": ['unranged'],
                                        "complex": [0], "isotop": [0], "charge": [0]})
display(range_data.style.applymap(ion_selection.display_color, subset=['color']))

,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,${}^{1}H^{+}$,1.010000,1.583612,1.503650,1.768199,#b2aa2d,H,1,1,+
1,${}^{27}Al^{2+}$,13.490000,13.409105,12.985088,14.254925,#e7e0d1,Al,1,27,2+
2,${}^{27}Al^{+}$,26.980000,26.993763,25.948003,28.963865,#e7e0d1,Al,1,27,+


In [24]:
change_color = widgets.Button(
    description='change color',
)
color_picker = widgets.ColorPicker(description='Select a color:')
row_index = widgets.IntText(value=0, description='index row:')
@change_color.on_click
def change_color_m(b,):
    with out:
        selected_color = mcolors.to_hex(color_picker.value)
        variables.range_data.at[row_index.value, 'color'] = selected_color
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))
        
# Create a container for the widgets
container = widgets.VBox([color_picker, row_index, change_color])
out = Output()
# Display the widgets
display(container, out)

Output()

$$\textbf{You can specify which dataset to use in below block}$$

In [29]:
# romove data that has x_det and y_det equal to zero at same time
# these points resulted in nan in the reconstruction becuse of dviding 0/0
# mask = np.logical_and((data['x_det (cm)'].to_numpy()==0), (data['y_det (cm)'].to_numpy()==0))
# print('The number of data with x=0 and y=0 in dataset is:', len(mask[mask==True]))
# data.drop(np.where(mask)[0], inplace=True)
# data.reset_index(inplace=True, drop=True)

The number of data with x=0 and y=0 in dataset is: 1


In [30]:
# exctract needed data from Pandas data frame as an numpy array
# dld_highVoltage = data['high_voltage (V)'].to_numpy()
# dld_x = data['x_det (cm)'].to_numpy()
# dld_y = data['y_det (cm)'].to_numpy()
# mc = data['mc_c (Da)'].to_numpy()

In [28]:
def mc_hist_plot(bin_size, lim_mc, ranging, figname):
    
    peaks_ini, peaks_y_ini, peak_widths_p_ini, _ = tools.hist_plot(variables.mc_c[variables.mc_c < lim_mc], variables, bin=bin_size, range_data=variables.range_data, ranging=ranging, adjust_label=False, plot=True, label='mc', fig_name=figname)
   
interact_manual(mc_hist_plot, bin_size=widgets.FloatText(value=0.1), lim_mc=widgets.IntText(value=400), figname=widgets.Text(value='hist'), ranging=widgets.Dropdown(options=[('True', True), ('False', False)]));

C:\Users\mehrp\AppData\Local\Temp\ipykernel_51244\2211174180.py:5: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=400, description='lim_mc'), …

In [29]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

In [33]:
avg_dens = element_selected.value[2]
field_evap = element_selected.value[3]

In [41]:
def x_y_z_calculation_and_plot(data, range_data, variables, element_percentage, kf, det_eff, icf, field_evap, avg_dens, rotary_fig_save, selected_are, figname):
    dld_highVoltage = data['high_voltage (V)'].to_numpy()
    dld_x = data['x_det (cm)'].to_numpy()
    dld_y = data['y_det (cm)'].to_numpy()
    mc = data['mc_c (Da)'].to_numpy()
    flight_path_length = flightPathLength.value
    px, py, pz = reconstruction.atom_probe_recons_from_detector_Gault_et_al(dld_x, dld_y, dld_highVoltage,
                                                                            flight_path_length, kf, det_eff, icf,
                                                                            field_evap, avg_dens)
    #     px, py, pz = reconstruction.atom_probe_recons_Bas_et_al(dld_x, dld_y, dld_highVoltage, flight_path_length, kf, det_eff, icfe, field_evap, avg_dens)
    data['x (nm)'] = px
    data['y (nm)'] = py
    data['z (nm)'] = pz
    reconstruction.reconstruction_plot(data, range_data, variables, element_percentage, rotary_fig_save, selected_are, figname)

In [ ]:
if range_data is not None:
    element_percentage = [0.01] * len(range_data['element'].tolist())
    element_percentage = str(element_percentage)
else:
    element_percentage = str([0.01])

interact_manual(x_y_z_calculation_and_plot, kf=widgets.FloatText(value=4), det_eff=widgets.FloatText(value=0.7), icf=widgets.FloatText(value=1.4),
                field_evap=widgets.FloatText(value=field_evap),
               avg_dens=widgets.FloatText(value=avg_dens), element_percentage=widgets.Textarea(value=element_percentage),
               rotary_fig_save=widgets.Dropdown(options=[('True', True), ('False', False)], value=False), data=fixed(data), range_data=fixed(range_data), variables=fixed(variables), selected_are=fixed(False), figname=widgets.Text(value='3d'));

In [ ]:
data

In [29]:

plot_3d_button = widgets.Button(
    description='plot 3D',
)
plot_heatmap_button = widgets.Button(
    description='plot heatmap',
)
plot_mc_button = widgets.Button(
    description='plot mc',
)
plot_projection_button = widgets.Button(
    description='plot projection',
)

clear_button = widgets.Button(
    description='Clear plots',
)

selected_area_p3=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
rotary_fig_save_p3=widgets.Dropdown(options=[('True', True), ('False', False)], description='Rotary save')
element_percentage_p3=widgets.Textarea(value=element_percentage, description='Element percentage')
@plot_3d_button.on_click
def plot_3d(b, data=data, range_data=range_data, figname=figname):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        reconstruction.reconstruction_plot(data, range_data, element_percentage_p3.value, rotary_fig_save_p3.value, selected_area_p3.value, figname)

selected_area_ph=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
element_percentage_ph=widgets.Textarea(value=element_percentage, description='Element percentage')
@plot_heatmap_button.on_click
def plot_heatmap(b, data=data, range_data=range_data):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_ph.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]

            data_f.reset_index(inplace=True, drop=True)
        reconstruction.heatmap(data_f, range_data, selected_area_ph.value, element_percentage_ph.value, save=True)

selected_area_pm=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
range_file_exist_pm=widgets.Dropdown(options=[('True', True), ('False', False)], description='Range exist')
bin_size_pm = widgets.FloatText(value=0.1, description='Bins size')
prominence_pm = widgets.IntText(value=50, description='Prominance')
distance_pm = widgets.IntText(value=100, description='Distance')
lim_mc_pm = widgets.IntText(value=150, description='Limit mc')
percent_pm = widgets.IntText(value=50, description='Percent MRP')
@plot_mc_button.on_click
def plot_mc(b, data=data, range_data=range_data):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_pm.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]

            data_f.reset_index(inplace=True, drop=True)
        mc = data_f['mc_c (Da)'].to_numpy()

        peaks_ini, peaks_y_ini, peak_widths_p_ini = tools.hist_plot(mc[mc < lim_mc_pm.value], range_data=range_data, ranging=range_file_exist_pm.value, bin=bin_size_pm.value, distance=distance_pm.value, percent=percent_pm.value,
                                                                    prominence=prominence_pm.value, selector=None, plot=True, label='mc', fig_name=figname+'_ini', fast_hist=False)
        index_max_ini = np.argmax(peaks_y_ini)
        mrp = (peaks_ini[index_max_ini] / (peak_widths_p_ini[index_max_ini][2] - peak_widths_p_ini[index_max_ini][1]))
        print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
        for i in range(len(peaks_ini)):
            print('Peaks ', i, 'is at location and height: ({:.2f}, {:.2f})'.format(peaks_ini[i], peaks_y_ini[i]), 'peak window sides (half-maximum) are: ({:.2f}, {:.2f})'.format(peak_widths_p_ini[i][1], peak_widths_p_ini[i][2]))

element_percentage_pp=widgets.Textarea(value=element_percentage, description='Element percentage')
selected_area_pp=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
x_or_y_pp=widgets.Dropdown(options=['x', 'y'], value='x', description='X or Y')

@plot_projection_button.on_click
def plot_projection(b, data=data, range_data=range_data, figname=figname):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_pp.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]
        
            data_f.reset_index(inplace=True, drop=True)
        reconstruction.projection(data_f, range_data, element_percentage_pp.value, selected_area_pp.value, x_or_y_pp.value, figname)

@clear_button.on_click
def clear(b,):
    with out:
        clear_output(True)
        print('')


tab1 = VBox(children=[selected_area_pp, x_or_y_pp, element_percentage_pp, plot_projection_button, clear_button])
tab2 = VBox(children=[selected_area_p3, rotary_fig_save_p3, element_percentage_p3, plot_3d_button, clear_button])
tab3 = VBox(children=[selected_area_pm, bin_size_pm, prominence_pm, distance_pm, lim_mc_pm, percent_pm, plot_mc_button, clear_button])
tab4 = VBox(children=[selected_area_ph, element_percentage_ph, plot_heatmap_button, clear_button])
# tab2 = VBox(children=[VBox(children=[HBox(children=[plot_3d_button, selected_area_p3, range_file_exist_p3, rotary_fig_save_p3, element_percentage_p3]),
#                                      HBox(children=[plot_heatmap_button, selected_area_ph, element_percentage_ph]),
#                                      HBox(children=[plot_mc_button, selected_area_pm, range_file_exist_pm, bin_size_pm, prominence_pm, distance_pm, lim_mc_pm, percent_pm]), 
#                                      HBox(children=[plot_projection_button, selected_area_pp, x_or_y_pp, element_percentage_pp])]), clear_button])


tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, 'projection')
tab.set_title(1, '3d plot')
tab.set_title(2, 'mc plot')
tab.set_title(3, 'heatmap plot')



display(VBox(children=[tab]))
out = Output()
display(out)

Output()

In [30]:
# save the new data
name_save_file = variables.result_path + '//' + dataset_name + '.h5'
data_tools.store_df_to_hdf(name_save_file, data, 'df')

In [31]:
# save data as epos file
epos = ccapt_tools.ccapt_to_epos(data, pulse_mode=pulse_mode.value, path=variables.result_path, name=dataset_name + '.epos')
pos = ccapt_tools.ccapt_to_pos(data, path=variables.result_path, name=dataset_name + '.pos')

In [32]:
# save data in csv format
data_tools.store_df_to_csv(data, variables.result_path + dataset_name + '.csv')